Preprocessing data:

In [497]:
record1 = "databases/record1.csv"
record2 = "databases/record2.csv"

**Record 2:**

In [498]:
top_crimes = ["BURGLARY", "ROBBERY", "THEFT", "MANU/DIST/DEL/DISP OR POSS W/I", "SEXUAL ASSAULT 1ST DEGREE", "FORGERY 2ND DEGREE", "POS CNTRL SUB EXCEPT MARIJUANA", "THEFT BY RECEIVING STOLEN PROP", "POSSESSION OF METHAMPHETAMINE", "DRIVING UNDER REVOKED LICENSE"]

master_dict = {}
list_directory = []
file_record2 = open(record2, "r")
counter = 0
for line in file_record2:
    # split by comma
    line = line.split(",")
    if counter == 0:
        # strip \ufeff and \n
        line[0] = line[0][1:]
        line[-1] = line[-1][0:-1]
        # removes indexes we don't need
        line.pop(1)
        line.pop(1)
        line.pop(1)
        line.pop(1)
        line.pop(1)
        line.pop(1)
        line.pop(2)
        line.pop(2)
        line.pop(3)
        line.pop(3)
        list_directory = line
    else:
        line[-1] = line[-1][0:-1]
        # removes indexes we don't need
        line.pop(1)
        line.pop(1)
        line.pop(1)
        line.pop(1)
        line.pop(1)
        line.pop(1)
        line.pop(2)
        line.pop(2)
        line.pop(3)
        line.pop(3)
        # strips leading and ending whitespace:
        for item in range(len(line)):
            line[item] = line[item].strip()
        # add to master dict if not in already
        if line[0] not in master_dict and line[1] in top_crimes:
            master_dict[line[0]] = line
        # if it is in already, remove the id from dict as it is a multiple offender or duplicate
        elif line[0] in master_dict:
            del master_dict[line[0]]
    counter += 1

**Record 1:**

In [499]:
file_record1 = open(record1, "r")
counter = 0
for line in file_record1:
    line = line.split(",")
    if counter == 0:
        # remove all unneeded, then remove \n
        line.pop(0)
        line.pop(0)
        line.pop(0)
        line.pop(0)
        line.pop(0)
        line.pop(0)
        line.pop(0)
        line.pop(0)
        line.pop(0)
        line.pop(5)
        line.pop(14)
        line[-1] = line[-1][0:-1]
        list_directory = list_directory + line
        list_directory.append("SENTENCE LENGTH")
    else:
        #print(counter)
        line[-1] = line[-1][0:-1]
        
        # strips leading and ending whitespace:
        for item in range(len(line)):
            line[item] = line[item].strip()
            
        # we only want the ones in the dictionary (first offenders)
        if line[0] in master_dict:
            # remove unneeded indexes, save id
            id = line.pop(0)
            line.pop(0)
            line.pop(0)
            line.pop(0)
            line.pop(0)
            line.pop(0)
            line.pop(0)
            line.pop(0)
            line.pop(0)
            line.pop(5)
            line.pop(14)
            
            # see if person has earliest possible release date (13) and sentence begin date (5)
            if line[13] == '' or line[5] == '' or line[13] == ' ' or line[5] == ' ':
                # remove from dictionary if they don't
                del master_dict[id]
            
            else:
                # change DOB to year
                if line[0] != '':
                    line[0] = line[0][-4:]
                # change sentence pardoned to year
                if line[4] != '':
                    line[4] = line[4][-4:]
                # change sentence begin date to year
                if line[5] != '':
                    line[5] = line[5][-4:]
                # change parol eligibility date to year
                if line[12] != '':
                    line[12] = line[12][-4:]
                # change earliest possible release date to year
                if line[13] != '':
                    line[13] = line[13][-4:]
                # change inst release date to year
                if line[14] != '':
                    line[14] = line[14][-4:]
                # changed parole date to year
                if line[19] != '' and line[19][0:7] != "PAROLED":
                    line[19] = line[19][-4:]
                    
                # add sentence length
                
                if line[13] == 'LFE' or line[13] == 'DTH':
                    line.append(line[13])
                elif line[5] == 'EARM':
                    line.append(line[5])
                else:
                    sent_length = int(line[13]) - int(line[5])
                    line.append(str(sent_length))
                    
                if line[1] != 'WHITE' and line[1] != 'BLACK':
                    line[1] = 'OTHER'
                    
                master_dict[id] = master_dict[id] + line
                
    counter += 1

In [500]:
to_remove = [] 
for entry in master_dict:
    if len(master_dict[entry]) != 26:
        to_remove.append(entry)
        
for num in to_remove:
    del(master_dict[num])

First we must split the data into training and testing sets; this function splits the data into a specified number of groups

In [501]:
from collections import defaultdict
import copy

def split_data(data, splits):
    group = 0
    results = defaultdict(list)
    targets = defaultdict(list)

    for entry in data:
        cur_entry = copy.deepcopy(data[entry])
        release = data[entry][17]
        results[group].append(cur_entry)
        targets[group].append(release)
        
        #increments group each time - don't need to randomize since the data was added to the dataset in order of sentence
        #begin date, so this will evenly distribute dates and essentially randomize everything else.
        group = 0 if group == splits-1 else group+1
        
    zipped = defaultdict(list)
        
    for num in range(splits):
        zipped[num] = zip(results[num], targets[num])
    
    return zipped

Now we will actually split the data and create the dataframes for our train and test sets.

In [502]:
import numpy as np
from sklearn import linear_model
from pandas import DataFrame
import pandas as pd

traininglist = []
testinglist = []

X = master_dict

group_dict = split_data(X, 5)

for group_num in range(1):
    trainSet = []
    for other_nums in range(5):
        if group_num != other_nums:
            trainSet += group_dict[other_nums+1]
    x_train, y_train = zip(*trainSet)
    x_test, y_test = zip(*group_dict[group_num])
    x_train = np.array(x_train)
    x_test = np.array(x_test)
    
df = DataFrame(x_train, columns = ['ID NUMBER', 'OFFENSE ARREST DESC', 'OFFENSE ATTEMPT DESC', 'COUNTY COMMITTED', 'DATE OF BIRTH', 'RACE DESC', 'GENDER', 'FACILITY', 'CURRENT SENTENCE PARDONED OR COMMUTED DATE', 'SENTENCE BEGIN DATE', 'MIN TERM/YEAR', 'MIN MONTH', 'MIN DAY', 'MAX TERM/YEAR', 'MAX MONTH', 'MAX DAY', 'PAROLE ELIGIBILITY DATE', 'EARLIEST POSSIBLE RELEASE DATE', 'INST RELEASE DATE', 'INST RELEASE TYPE', 'PAROLE BOARD NEXT REVIEW DATE(MONTH&YEAR)', 'PAROLE BOARD FINAL HEARING DATE(MONTH&YEAR)', 'PAROLE BOARD STATUS', 'PAROLE DATE', 'PAROLE DISCHARGE DESC', 'SENTENCE LENGTH'])
df_test = DataFrame(x_test, columns = ['ID NUMBER', 'OFFENSE ARREST DESC', 'OFFENSE ATTEMPT DESC', 'COUNTY COMMITTED', 'DATE OF BIRTH', 'RACE DESC', 'GENDER', 'FACILITY', 'CURRENT SENTENCE PARDONED OR COMMUTED DATE', 'SENTENCE BEGIN DATE', 'MIN TERM/YEAR', 'MIN MONTH', 'MIN DAY', 'MAX TERM/YEAR', 'MAX MONTH', 'MAX DAY', 'PAROLE ELIGIBILITY DATE', 'EARLIEST POSSIBLE RELEASE DATE', 'INST RELEASE DATE', 'INST RELEASE TYPE', 'PAROLE BOARD NEXT REVIEW DATE(MONTH&YEAR)', 'PAROLE BOARD FINAL HEARING DATE(MONTH&YEAR)', 'PAROLE BOARD STATUS', 'PAROLE DATE', 'PAROLE DISCHARGE DESC', 'SENTENCE LENGTH'])

Now we will process the data to make categorical values numerical and engineer some features. 

In [503]:
from sklearn import preprocessing

#Make gender numerical
le = preprocessing.LabelEncoder()
le.fit(['FEMALE','MALE'])
df['GENDER'] = le.transform(df['GENDER'])
df_test['GENDER'] = le.transform(df_test['GENDER'])

In [504]:
#create dummy variables for race (for guessing sentence length)
dummy = pd.get_dummies(df['RACE DESC'])
df = pd.concat([df, dummy], axis=1)

dummy = pd.get_dummies(df_test['RACE DESC'])
df_test = pd.concat([df_test, dummy], axis=1)

#make race numerical
le.fit(['WHITE','BLACK','OTHER'])
df['RACE DESC'] = le.transform(df['RACE DESC'])
df_test['RACE DESC'] = le.transform(df_test['RACE DESC'])

In [505]:
#create dummy variables for offenses
dummy = pd.get_dummies(df['OFFENSE ARREST DESC'])
for crime in dummy:
    if crime in top_crimes:
        df = pd.concat([df, dummy[crime]], axis=1)

dummy = pd.get_dummies(df_test['OFFENSE ARREST DESC'])
for crime in dummy:
    if crime in top_crimes:
        df_test = pd.concat([df_test, dummy[crime]], axis=1)

In [506]:
#fix sentences to all be numerical
fixed_sentences = []
for sentence in df['SENTENCE LENGTH']:
    if sentence == 'LFE' or sentence == 'DTH':
        fixed_sentences.append(122)
    else:
        fixed_sentences.append((int)(sentence))
        
df['SENTENCE LENGTH'] = fixed_sentences

fixed_sentences = []

for sentence in df_test['SENTENCE LENGTH']:
    if sentence == 'LFE' or sentence == 'DTH':
        fixed_sentences.append(122)
    else:
        fixed_sentences.append((int)(sentence))
        
df_test['SENTENCE LENGTH'] = fixed_sentences

#fix min terms to all be numerical
fixed_sentences = []
for sentence in df['MIN TERM/YEAR']:
    if sentence == 'LFE' or sentence == 'DTH':
        fixed_sentences.append(122)
    elif sentence == 'IND':
        fixed_sentences.append(0)
    else:
        fixed_sentences.append(sentence)
        
df['MIN TERM/YEAR'] = fixed_sentences
fixed_sentences = []

for sentence in df_test['MIN TERM/YEAR']:
    if sentence == 'LFE' or sentence == 'DTH':
        fixed_sentences.append(122)
    elif sentence == 'IND':
        fixed_sentences.append(0)
    else:
        fixed_sentences.append(sentence)
        
df_test['MIN TERM/YEAR'] = fixed_sentences

#fix max terms to all be numerical
fixed_sentences = []
for sentence in df['MAX TERM/YEAR']:
    if sentence == 'LFE' or sentence == 'DTH':
        fixed_sentences.append(122)
    elif sentence == 'IND':
        fixed_sentences.append(0)
    else:
        fixed_sentences.append(sentence)
        
df['MAX TERM/YEAR'] = fixed_sentences
fixed_sentences = []

for sentence in df_test['MAX TERM/YEAR']:
    if sentence == 'LFE' or sentence == 'DTH':
        fixed_sentences.append(122)
    elif sentence == 'IND':
        fixed_sentences.append(0)
    else:
        fixed_sentences.append(sentence)
        
df_test['MAX TERM/YEAR'] = fixed_sentences

In [507]:
#engineer two features - sentence length difference from min term and from max term
engineered_feature = []
for i in range(len(df['SENTENCE LENGTH'])):
    engineered_feature.append((int)(df['SENTENCE LENGTH'][i])-(int)(df['MIN TERM/YEAR'][i]))
df['SENTENCE WRT MIN TERM'] = engineered_feature

engineered_feature = []
for i in range(len(df_test['SENTENCE LENGTH'])):
    engineered_feature.append((int)(df_test['SENTENCE LENGTH'][i])-(int)(df_test['MIN TERM/YEAR'][i]))
df_test['SENTENCE WRT MIN TERM'] = engineered_feature

engineered_feature = []
for i in range(len(df['SENTENCE LENGTH'])):
    engineered_feature.append((int)(df['MAX TERM/YEAR'][i])-(int)(df['SENTENCE LENGTH'][i]))
df['SENTENCE WRT MAX TERM'] = engineered_feature

engineered_feature = []
for i in range(len(df_test['SENTENCE LENGTH'])):
    engineered_feature.append((int)(df_test['MAX TERM/YEAR'][i])-(int)(df_test['SENTENCE LENGTH'][i]))
df_test['SENTENCE WRT MAX TERM'] = engineered_feature

        
df.head()

,ID NUMBER,OFFENSE ARREST DESC,OFFENSE ATTEMPT DESC,COUNTY COMMITTED,DATE OF BIRTH,RACE DESC,GENDER,FACILITY,CURRENT SENTENCE PARDONED OR COMMUTED DATE,SENTENCE BEGIN DATE,...,FORGERY 2ND DEGREE,MANU/DIST/DEL/DISP OR POSS W/I,POS CNTRL SUB EXCEPT MARIJUANA,POSSESSION OF METHAMPHETAMINE,ROBBERY,SEXUAL ASSAULT 1ST DEGREE,THEFT,THEFT BY RECEIVING STOLEN PROP,SENTENCE WRT MIN TERM,SENTENCE WRT MAX TERM
0,23916,ROBBERY,,GAGE,1938,2,1,NEBRASKA STATE PENITENTIARY,,1964,...,0,0,0,0,1,0,0,0,12,-8
1,26652,SEXUAL ASSAULT 1ST DEGREE,,DOUGLAS,1951,2,1,NEBRASKA STATE PENITENTIARY,,1969,...,0,0,0,0,0,1,0,0,1,9
2,27041,BURGLARY,,DOUGLAS,1937,1,1,NEBRASKA STATE PENITENTIARY,,1969,...,0,0,0,0,0,0,0,0,7,-2
3,27238,BURGLARY,,DOUGLAS,1949,2,1,NEBRASKA STATE PENITENTIARY,,1970,...,0,0,0,0,0,0,0,0,6,0
4,27394,ROBBERY,,SARPY,1950,2,1,NEBRASKA STATE PENITENTIARY,,1973,...,0,0,0,0,1,0,0,0,2,9


In [513]:
from sklearn.ensemble import RandomForestClassifier
import random

#function to predict race based on given factors and print stats
def predict(df, df_test):
    X = df[['GENDER',
           'BURGLARY', 'DRIVING UNDER REVOKED LICENSE', 'FORGERY 2ND DEGREE',
           'MANU/DIST/DEL/DISP OR POSS W/I', 'POS CNTRL SUB EXCEPT MARIJUANA',
           'POSSESSION OF METHAMPHETAMINE', 'ROBBERY', 'SEXUAL ASSAULT 1ST DEGREE',
           'THEFT', 'THEFT BY RECEIVING STOLEN PROP','SENTENCE WRT MIN TERM','SENTENCE WRT MAX TERM']]
    Y = df['RACE DESC']
    TestX = df_test[['GENDER',
           'BURGLARY', 'DRIVING UNDER REVOKED LICENSE', 'FORGERY 2ND DEGREE',
           'MANU/DIST/DEL/DISP OR POSS W/I', 'POS CNTRL SUB EXCEPT MARIJUANA',
           'POSSESSION OF METHAMPHETAMINE', 'ROBBERY', 'SEXUAL ASSAULT 1ST DEGREE',
           'THEFT', 'THEFT BY RECEIVING STOLEN PROP','SENTENCE WRT MIN TERM','SENTENCE WRT MAX TERM']]

    random.seed(100)
    rf = RandomForestClassifier(n_estimators=1000)
    rf.fit(X,Y)
    out = rf.predict(TestX)
    
    #calculate percent correct
    matches = 0
    naive = 0
    i = 0

    for num in df_test['RACE DESC']:
        if num == out[i]:
            matches += 1
        if num == 2:
            naive += 1
        i += 1

    print("Percent Correct:", matches/i)
    print("Percent Correct with naive model:", naive/i)

    #get top features
    names = ['GENDER',
           'BURGLARY', 'DRIVING UNDER REVOKED LICENSE', 'FORGERY 2ND DEGREE',
           'MANU/DIST/DEL/DISP OR POSS W/I', 'POS CNTRL SUB EXCEPT MARIJUANA',
           'POSSESSION OF METHAMPHETAMINE', 'ROBBERY', 'SEXUAL ASSAULT 1ST DEGREE',
           'THEFT', 'THEFT BY RECEIVING STOLEN PROP','SENTENCE WRT MIN TERM','SENTENCE WRT MAX TERM']

    feature_ranking = sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), names), 
                 reverse=True)
    print("Top features:", list(map(lambda x: (x[1] + ": " + (str)(x[0])), feature_ranking[0:3])))

In [521]:
import statistics

#function to predict sentence length based on given factors and print results
def predict_years(df, df_test):
    X = df[['GENDER','WHITE','BLACK','OTHER',
           'BURGLARY', 'DRIVING UNDER REVOKED LICENSE', 'FORGERY 2ND DEGREE',
           'MANU/DIST/DEL/DISP OR POSS W/I', 'POS CNTRL SUB EXCEPT MARIJUANA',
           'POSSESSION OF METHAMPHETAMINE', 'ROBBERY', 'SEXUAL ASSAULT 1ST DEGREE',
           'THEFT', 'THEFT BY RECEIVING STOLEN PROP','SENTENCE WRT MIN TERM','SENTENCE WRT MAX TERM']]
    Y = df['SENTENCE WRT MAX TERM']
    TestX = df_test[['GENDER','WHITE','BLACK','OTHER',
           'BURGLARY', 'DRIVING UNDER REVOKED LICENSE', 'FORGERY 2ND DEGREE',
           'MANU/DIST/DEL/DISP OR POSS W/I', 'POS CNTRL SUB EXCEPT MARIJUANA',
           'POSSESSION OF METHAMPHETAMINE', 'ROBBERY', 'SEXUAL ASSAULT 1ST DEGREE',
           'THEFT', 'THEFT BY RECEIVING STOLEN PROP','SENTENCE WRT MIN TERM','SENTENCE WRT MAX TERM']]

    random.seed(100)
    rf = RandomForestClassifier(n_estimators=1000)
    rf.fit(X,Y)
    out = rf.predict(TestX)
    matches = 0
    avg = 0
    naive_mean = 0
    naive_median = 0
    naive_1 = 0
    i = 0

    #calculate average distance from actual value and test various naive models
    for num in df_test['SENTENCE WRT MAX TERM']:
        matches += abs(num-out[i])
        avg += num
        i += 1
        
    avg /= i
    i = 0
    for num in df_test['SENTENCE WRT MAX TERM']:
        naive_mean += abs(avg-out[i])
        i += 1
    
    i = 0
    median = statistics.median(df_test['SENTENCE WRT MAX TERM'])
    for num in df_test['SENTENCE WRT MAX TERM']:
        naive_median += abs(median-out[i])
        i += 1
        
    i = 0
    for num in df_test['SENTENCE WRT MAX TERM']:
        naive_1 += abs(1-out[i])
        i += 1
    

    print("Average Years Off:", matches/i)
    print("Average Years Off with naive model using mean:", naive_mean/i)
    print("Average Years Off with naive model using median:", naive_median/i)
    print("Average Years Off with naive model using mode:", naive_0/i)

    
    #find top features
    names = ['GENDER','WHITE','BLACK','OTHER',
           'BURGLARY', 'DRIVING UNDER REVOKED LICENSE', 'FORGERY 2ND DEGREE',
           'MANU/DIST/DEL/DISP OR POSS W/I', 'POS CNTRL SUB EXCEPT MARIJUANA',
           'POSSESSION OF METHAMPHETAMINE', 'ROBBERY', 'SEXUAL ASSAULT 1ST DEGREE',
           'THEFT', 'THEFT BY RECEIVING STOLEN PROP','SENTENCE WRT MIN TERM','SENTENCE WRT MAX TERM']

    feature_ranking = sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), names), 
                 reverse=True)
    print("Top features:", list(map(lambda x: (x[1] + ": " + (str)(x[0])), feature_ranking[0:3])))

In [515]:
#print race prediction results on the training and testing sets for all crimes and each crime
print("All Crimes:")
print("\nTraining Set Results:")
predict(df, df)
print("\nTesting Set Results:")
predict(df, df_test)

print("\n\nBurglary:")
df_burglary = df[df["OFFENSE ARREST DESC"] == "BURGLARY"]
df_burglary_test = df_test[df_test["OFFENSE ARREST DESC"] == "BURGLARY"]
print("\nTraining Set Results:")
predict(df_burglary, df_burglary)
print("\nTesting Set Results:")
predict(df_burglary, df_burglary_test)

print("\n\nDriving Under Revoked License:")
df_driving = df[df["OFFENSE ARREST DESC"] == "DRIVING UNDER REVOKED LICENSE"]
df_driving_test = df_test[df_test["OFFENSE ARREST DESC"] == "DRIVING UNDER REVOKED LICENSE"]
print("\nTraining Set Results:")
predict(df_driving, df_driving)
print("\nTesting Set Results:")
predict(df_driving, df_driving_test)

print("\n\nForgery 2nd Degree:")
df_forgery = df[df["OFFENSE ARREST DESC"] == "FORGERY 2ND DEGREE"]
df_forgery_test = df_test[df_test["OFFENSE ARREST DESC"] == "FORGERY 2ND DEGREE"]
print("\nTraining Set Results:")
predict(df_forgery, df_forgery)
print("\nTesting Set Results:")
predict(df_forgery, df_forgery_test)

print("\n\nPossession With Intent to Distribute and Similar:")
df_dist = df[df["OFFENSE ARREST DESC"] == "MANU/DIST/DEL/DISP OR POSS W/I"]
df_dist_test = df_test[df_test["OFFENSE ARREST DESC"] == "MANU/DIST/DEL/DISP OR POSS W/I"]
print("\nTraining Set Results:")
predict(df_dist, df_dist)
print("\nTesting Set Results:")
predict(df_dist, df_dist_test)

print("\n\nPossession of Controlled Substance Except Marijuana:")
df_possession = df[df["OFFENSE ARREST DESC"] == "POS CNTRL SUB EXCEPT MARIJUANA"]
df_possession_test = df_test[df_test["OFFENSE ARREST DESC"] == "POS CNTRL SUB EXCEPT MARIJUANA"]
print("\nTraining Set Results:")
predict(df_possession, df_possession)
print("\nTesting Set Results:")
predict(df_possession, df_possession_test)

print("\n\nPossession of Methamphetamine:")
df_meth = df[df["OFFENSE ARREST DESC"] == "POSSESSION OF METHAMPHETAMINE"]
df_meth_test = df_test[df_test["OFFENSE ARREST DESC"] == "POSSESSION OF METHAMPHETAMINE"]
print("\nTraining Set Results:")
predict(df_meth, df_meth)
print("\nTesting Set Results:")
predict(df_meth, df_meth_test)

print("\n\nRobbery:")
df_robbery = df[df["OFFENSE ARREST DESC"] == "ROBBERY"]
df_robbery_test = df_test[df_test["OFFENSE ARREST DESC"] == "ROBBERY"]
print("\nTraining Set Results:")
predict(df_robbery, df_robbery)
print("\nTesting Set Results:")
predict(df_robbery, df_robbery_test)

print("\n\nSexual Assault 1st Degree:")
df_assault = df[df["OFFENSE ARREST DESC"] == "SEXUAL ASSAULT 1ST DEGREE"]
df_assault_test = df_test[df_test["OFFENSE ARREST DESC"] == "SEXUAL ASSAULT 1ST DEGREE"]
print("\nTraining Set Results:")
predict(df_assault, df_assault)
print("\nTesting Set Results:")
predict(df_assault, df_assault_test)

print("\n\nTheft:")
df_theft = df[df["OFFENSE ARREST DESC"] == "THEFT"]
df_theft_test = df_test[df_test["OFFENSE ARREST DESC"] == "THEFT"]
print("\nTraining Set Results:")
predict(df_theft, df_theft)
print("\nTesting Set Results:")
predict(df_theft, df_theft_test)

print("\n\nTheft By Receiving Stolen Property:")
df_stolen = df[df["OFFENSE ARREST DESC"] == "THEFT BY RECEIVING STOLEN PROP"]
df_stolen_test = df_test[df_test["OFFENSE ARREST DESC"] == "THEFT BY RECEIVING STOLEN PROP"]
print("\nTraining Set Results:")
predict(df_stolen, df_stolen)
print("\nTesting Set Results:")
predict(df_stolen, df_stolen_test)

All Crimes:

Training Set Results:
Percent Correct: 0.6413697362332254
Percent Correct with naive model: 0.5908375751966682
Top features: ['SENTENCE WRT MAX TERM: 0.4143', 'SENTENCE WRT MIN TERM: 0.3025', 'ROBBERY: 0.0977']

Testing Set Results:
Percent Correct: 0.5941143808995003
Percent Correct with naive model: 0.5893947806774015
Top features: ['SENTENCE WRT MAX TERM: 0.4121', 'SENTENCE WRT MIN TERM: 0.3022', 'ROBBERY: 0.0998']


Burglary:

Training Set Results:
Percent Correct: 0.6594696969696969
Percent Correct with naive model: 0.6431818181818182
Top features: ['SENTENCE WRT MAX TERM: 0.5511', 'SENTENCE WRT MIN TERM: 0.4198', 'GENDER: 0.029']

Testing Set Results:
Percent Correct: 0.6407879490150638
Percent Correct with naive model: 0.6500579374275782
Top features: ['SENTENCE WRT MAX TERM: 0.5513', 'SENTENCE WRT MIN TERM: 0.421', 'GENDER: 0.0277']


Driving Under Revoked License:

Training Set Results:
Percent Correct: 0.6483870967741936
Percent Correct with naive model: 0.632258

In [522]:
#print sentence length prediction results on the training and testing sets for all crimes and each crime
print("All Crimes:")
print("\nTraining Set Results:")
predict_years(df, df)
print("\nTesting Set Results:")
predict_years(df, df_test)

print("\n\nBurglary:")
print("\nTraining Set Results:")
predict_years(df_burglary, df_burglary)
print("\nTesting Set Results:")
predict_years(df_burglary, df_burglary_test)

print("\n\nDriving Under Revoked License:")
print("\nTraining Set Results:")
predict_years(df_driving, df_driving)
print("\nTesting Set Results:")
predict_years(df_driving, df_driving_test)

print("\n\nForgery 2nd Degree:")
print("\nTraining Set Results:")
predict_years(df_forgery, df_forgery)
print("\nTesting Set Results:")
predict_years(df_forgery, df_forgery_test)

print("\n\nPossession With Intent to Distribute and Similar:")
print("\nTraining Set Results:")
predict_years(df_dist, df_dist)
print("\nTesting Set Results:")
predict_years(df_dist, df_dist_test)

print("\n\nPossession of Controlled Substance Except Marijuana:")
print("\nTraining Set Results:")
predict_years(df_possession, df_possession)
print("\nTesting Set Results:")
predict_years(df_possession, df_possession_test)

print("\n\nPossession of Methamphetamine:")
print("\nTraining Set Results:")
predict_years(df_meth, df_meth)
print("\nTesting Set Results:")
predict_years(df_meth, df_meth_test)

print("\n\nRobbery:")
print("\nTraining Set Results:")
predict_years(df_robbery, df_robbery)
print("\nTesting Set Results:")
predict_years(df_robbery, df_robbery_test)

print("\n\nSexual Assault 1st Degree:")
print("\nTraining Set Results:")
predict_years(df_assault, df_assault)
print("\nTesting Set Results:")
predict_years(df_assault, df_assault_test)

print("\n\nTheft:")
print("\nTraining Set Results:")
predict_years(df_theft, df_theft)
print("\nTesting Set Results:")
predict_years(df_theft, df_theft_test)

print("\n\nTheft By Receiving Stolen Property:")
print("\nTraining Set Results:")
predict_years(df_stolen, df_stolen)
print("\nTesting Set Results:")
predict_years(df_stolen, df_stolen_test)

All Crimes:

Training Set Results:
Average Years Off: 0.0
Average Years Off with naive model using mean: 1.9016595270027146
Average Years Off with naive model using median: 1.7761221656640445
Average Years Off with naive model using mode: 0.3961129106894956
Top features: ['SENTENCE WRT MAX TERM: 0.9005', 'SENTENCE WRT MIN TERM: 0.0659', 'SEXUAL ASSAULT 1ST DEGREE: 0.0067']

Testing Set Results:
Average Years Off: 0.1246529705719045
Average Years Off with naive model using mean: 1.7766893955205616
Average Years Off with naive model using median: 1.6610216546363132
Average Years Off with naive model using mode: 1.1882287617990006
Top features: ['SENTENCE WRT MAX TERM: 0.9014', 'SENTENCE WRT MIN TERM: 0.065', 'SEXUAL ASSAULT 1ST DEGREE: 0.0067']


Burglary:

Training Set Results:
Average Years Off: 0.0
Average Years Off with naive model using mean: 1.5734561524334287
Average Years Off with naive model using median: 1.5454545454545454
Average Years Off with naive model using mode: 1.621212